In [32]:
import pandas as pd

from agage_archive import Paths
from agage_archive.io import combine_datasets, read_agage, read_ale_gage, output_dataset

path = Paths()

In [43]:
instrument = "GCMD"

with open(path.root / "data/data_selection/data_release_schedule.xlsx", "+rb") as f:
    df_all = pd.read_excel(f, sheet_name=instrument)

    # Get index of row that contains "General release date"
    idx = df_all[df_all.iloc[:, 0] == "General release date"].index[0]
    general_end_date = df_all.iloc[idx, 1]

    df = pd.read_excel(f, sheet_name="GCMD", skiprows=idx+2)

# Remove whitespace
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# If NaN in df, replace with general end date
df = df.fillna(general_end_date)

df.set_index("Species", inplace=True)

In [44]:
# Process for all species and sites
for species in df.index:
    for site in df.columns:
        if df.loc[species, site] != "x":
            ds = read_agage(species, site, instrument)
            output_dataset(ds, "AGAGE", instrument="GCMD", end_date=df.loc[species, site])